daisyRec version 1.1.2

In [1]:
import daisy.utils

In [2]:
from daisy.utils.metrics import *

In [3]:
import pandas as pd

In [4]:
df_pred = pd.read_csv('../pred.csv')

df_test = pd.read_csv('../test.csv')

df_pred_new = pd.merge(
    df_pred, 
    df_test.loc[:, ['user_id', 'item_id', 'relevance']], on=['user_id', 'item_id'], how='left'
)

df_pred_new = df_pred_new[df_pred_new.user_id.isin(df_test.user_id)]

df_pred_new.fillna(0, inplace=True)

df_pred_new.relevance.value_counts()

0.0    51516
4.5     1840
5.0     1304
Name: relevance, dtype: int64

In [5]:
df_pred_new.head()

,user_id,item_id,score,relevance
40,3,590,0.280305,0.0
41,3,1136,0.263707,0.0
42,3,920,0.231234,0.0
43,3,1246,0.188023,0.0
44,3,318,0.187111,0.0


In [6]:
df_test.head()

,user_id,item_id,rating,timestamp,relevance
0,1256,5378,5.0,2007-10-31 12:18:24,5.0
1,1256,778,4.5,2007-10-31 12:19:51,4.5
2,1256,8376,5.0,2007-10-31 12:30:59,5.0
3,1256,2594,5.0,2007-10-31 12:20:49,5.0
4,1256,30810,4.5,2007-10-31 12:32:24,4.5


In [7]:
recommendations = {}

for user, relevance in zip(df_pred_new.user_id, df_pred_new.relevance):
    if user in recommendations:
        recommendations[user] += [relevance]
    else:
        recommendations[user] = [relevance]

In [8]:
gt_recommendations = {}

for user, item_id in zip(df_test.user_id, df_test.item_id):
    if user in gt_recommendations:
        gt_recommendations[user] += [item_id]
    else:
        gt_recommendations[user] = [item_id] 

In [9]:
map_at_k(recommendations_binary)

NameError: name 'recommendations_binary' is not defined

Let's fix this small bug:

In [10]:
def map_at_k(rs):
    """
    Mean Average Precision calculation method
    Parameters
    ----------
    rs : Dict, {user : rank items} for test set
    Returns
    -------
    m_a_p : float, MAP value
    """
    m_a_p = np.mean([ap(r) for r in rs.values()])
    return m_a_p

In [11]:
map_at_k(recommendations)

0.022970972145948697

In [12]:
hr_at_k(recommendations, gt_recommendations)

0.47457006952067327

In [13]:
np.mean([precision_at_k(x, 20) for x in list(recommendations.values())])

0.057519209659714604

In [14]:
df_merged = pd.merge(df_test, df_pred_new, on=('user_id', 'item_id'), how='outer')
df_merged = df_merged.sort_values('score')
df_merged.relevance_x.fillna(0, inplace=True)

In [15]:
np.mean([
    ndcg_at_k(
        df_merged[df_merged.user_id == user].relevance_x.tolist(), 
        20
    ) 
    for user in recommendations
])

0.0730865703295614

In [16]:
recall_at_k(recommendations, gt_recommendations, 20)

0.09632131280310641

In [17]:
mrr_at_k(recommendations, 20)

0.2748816881385204

In [18]:
auc_at_k(recommendations)

ZeroDivisionError: float division by zero

AUC raises an error if pos_num or neg_num is 0

There may be some variations in the AP formula:

In [19]:
def ap(r, user):
    """
    Average precision calculation method
    Parameters
    ----------
    r : List, Relevance scores (list or numpy) in rank order (first element is the first item)
    Returns
    -------
    a_p : float, Average precision value
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    a_p = np.sum(out) / len(gt_recommendations[user])

    return a_p


def map_at_k(rs):
    """
    Mean Average Precision calculation method
    Parameters
    ----------
    rs : Dict, {user : rank items} for test set
    Returns
    -------
    m_a_p : float, MAP value
    """
    
    return np.mean([ap(r, user) for user, r in rs.items()])

In [20]:
map_at_k(recommendations)

0.031690245847362415

In [21]:
def ap(r, user):
    """
    Average precision calculation method
    Parameters
    ----------
    r : List, Relevance scores (list or numpy) in rank order (first element is the first item)
    Returns
    -------
    a_p : float, Average precision value
    """
    r = np.asarray(r) != 0
    out = [r[k] * precision_at_k(r, k + 1) for k in range(r.size)]
#     if not out:
#         return 0.

#     max_good = len(tmp_test[user])
#     max_good = len(r)
    max_good = min(len(gt_recommendations[user]), len(r))
    a_p = np.sum(out) / max_good

    return a_p

In [22]:
map_at_k(recommendations)

0.03927176078866248